In [1]:
#import library and module
import pandas as pd
from resume_screening import resparser, match

c:\Users\Acer\anaconda3\envs\resume-parser\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
from nltk.corpus import stopwords
stopw  = set(stopwords.words('english'))

In [3]:
#read indeed job vacancy data
job = pd.read_csv('indeed_data.csv')
job['test'] = job['description'].apply(lambda x: ' '.join([word for word in str(x).split() if len(word)>2 and word not in (stopw)]))

In [4]:
df = job.drop_duplicates(subset='test').reset_index(drop=True)
df.head()

,title,company,salary,description,link,test
0,Data Scientist (Analytics),Grab,None,"Job Description: Life at Grab At Grab, every G...",https://id.indeed.com/rc/clk?jk=acd6479660d3fc...,"Job Description: Life Grab Grab, every Grabber..."
1,Data Scientist,Sayurbox,None,Job Description Be a part of Sayurbox data tea...,https://id.indeed.com/rc/clk?jk=40482050a5e4d8...,Job Description part Sayurbox data team target...
2,Senior Data Scientist - GoPay,GO-JEK,None,Location Jakarta Work Type Permanent Applicati...,https://id.indeed.com/rc/clk?jk=60463a42bb19a6...,Location Jakarta Work Type Permanent Applicati...
3,Data Scientist,Allianz Indonesia,None,Bachelor Degree with field of study mathematic...,https://id.indeed.com/rc/clk?jk=835f2c980867fe...,"Bachelor Degree field study mathematic, inform..."
4,Senior Data Scientist,Sayurbox,None,Job Description: Build Sayurbox data team acco...,https://id.indeed.com/rc/clk?jk=da6afcc2ee471d...,Job Description: Build Sayurbox data team acco...


In [5]:
test = (df['test'].values.astype('U'))

In [6]:
print("There are {} job vacancy list scrapped from id.indeed.com".format(df.shape[0]))

There are 567 job vacancy list scrapped from id.indeed.com


In [7]:
#input the resume or CV file
filled = input()
skills = resparser.skill("resume_files/{}".format(filled))

In [8]:
skills.append(match.preprocessing(skills[0]))
del skills[0]

print("SKILLS LIST:\n",skills)

SKILLS LIST:
 ['web promotional teaching export ug investors ux user interface design training graphic design google building mobile applications usability mobile manuals organization moc experience design interview digital illustration prototype space web design design dsc gsm visual design user experience design app instagram sketching ux design options usability testing higher education feeds mastery online interfaces java internships developers ued banners social media figma machine learning materials learning linkedin testing user experience content design wireframing media design']


# Model KNN - Minkowski Metric

In [9]:
#Train the KNN model using Minkowski Metric
matches = match.knearestNeighbors(skills, test)
matches = pd.DataFrame(matches, columns=['Match confidence'])

In [11]:
#Job Vacancy Recommendation
result_minkowski = df[['title','company','salary']].copy()
result_minkowski['match'] = matches['Match confidence'].copy()
result_minkowski = result_minkowski.sort_values('match').reset_index(drop=True).head(20)
result_minkowski

,title,company,salary,match
0,UX Designer,PT HZN Teknologi Indonesia (Horizon),None,0.54
1,UX/UI Designer (Ref. No. 2156),Global Landscapes Forum (CIFOR),None,0.56
2,UI Designer - Internship,PT HZN Teknologi Indonesia (Horizon),None,0.57
3,Senior UI/UX Designer,Tokenomy,None,0.57
4,UI/ UX Designer,Tinkerlust,None,0.57
5,UX / UI Designer,Startups,None,0.58
6,Senior UI/UX Designer,Cermati.com,None,0.59
7,UI / UX Designer,PT Mobile Sarana Sentosa,None,0.60
8,UI/UX Designer,Mekari (PT. Mid Solusi Nusantara),None,0.61
9,Senior UI/UX Designer,Whello Indonesia,None,0.61


# Model Cosine Similarity

In [12]:
df['clean'] = df['test'].apply(match.preprocessing)
jobdesc = (df['clean'].values.astype('U'))

In [13]:
count_matrix = match.vectorizing(skills[0], jobdesc)
matchPercentage = match.coSim(count_matrix)
matchPercentage = pd.DataFrame(matchPercentage, columns=['Skills Match'])

In [16]:
#Job Vacancy Recommendations
result_cosine = df[['title','company','salary']]
result_cosine = result_cosine.join(matchPercentage)
result_cosine = result_cosine.sort_values('Skills Match', ascending=False).reset_index(drop=True).head(20)
result_cosine

,title,company,salary,Skills Match
0,UI Designer - Internship,PT HZN Teknologi Indonesia (Horizon),None,68.50
1,UX Designer,PT HZN Teknologi Indonesia (Horizon),None,68.12
2,Senior UI/UX Designer,Whello Indonesia,None,56.29
3,Senior UI/UX Designer,"DOKU, PT NUSA SATU INTI ARTHA",None,55.31
4,UI / UX Designer,PT Mobile Sarana Sentosa,None,54.49
5,UI/UX DESIGNER,Dipstrategy,None,54.38
6,UI/ UX Designer,Tinkerlust,None,52.51
7,Senior UI/UX Designer,PT Nusa Satu Inti Artha,None,52.49
8,UI/UX Designer,Dekoruma,None,50.85
9,UI/UX Designer,Nomura Research Institute Indonesia,None,48.13


# Comparing between Minkowski Metric vs Cosine Similarity

In [17]:
data = [result_minkowski[['title','company']], result_cosine[['title','company']]]
headers = ['Minkowski','Cosine Similarity']
df_compare = pd.concat(data, axis = 1, keys = headers)
df_compare

Minkowski  \
                                        title   
0                                 UX Designer   
1              UX/UI Designer (Ref. No. 2156)   
2                    UI Designer - Internship   
3                       Senior UI/UX Designer   
4                             UI/ UX Designer   
5                            UX / UI Designer   
6                       Senior UI/UX Designer   
7                            UI / UX Designer   
8                              UI/UX Designer   
9                       Senior UI/UX Designer   
10                   Associate UI/UX Designer   
11                      Senior UI/UX Designer   
12                    UI/UX Designer (Senior)   
13                      Senior UI/UX Designer   
14                             UI/UX Designer   
15                             UI/UX Designer   
16                             UI/UX Designer   
17                             UI/UX DESIGNER   
18                             UI/UX Designer   
19  B2C Support - Creative and UI/UX Designer   

                                                 Cosine Similarity  \
                                 company                     title   
0   PT HZN Teknologi Indonesia (Horizon)  UI Designer - Internship   
1        Global Landscapes Forum (CIFOR)               UX Designer   
2   PT HZN Teknologi Indonesia (Horizon)     Senior UI/UX Designer   
3                               Tokenomy     Senior UI/UX Designer   
4                             Tinkerlust          UI / UX Designer   
5                               Startups            UI/UX DESIGNER   
6                            Cermati.com           UI/ UX Designer   
7               PT Mobile Sarana Sentosa     Senior UI/UX Designer   
8      Mekari (PT. Mid Solusi Nusantara)            UI/UX Designer   
9                       Whello Indonesia            UI/UX Designer   
10                               Devhaus     Senior UI/UX Designer   
11               PT Nusa Satu Inti Artha            UI/UX Designer   
12                              Pensieve            UI/UX Designer   
13         DOKU, PT NUSA SATU INTI ARTHA            UI/UX Designer   
14                  Bali Staff Solutions                 UX Writer   
15                             sribu.com          UX / UI Designer   
16                         1001malam.com  UI/UX Designer (Officer)   
17                           Dipstrategy            UI/UX Designer   
18               Bintoro Sinergi Korpora            UI/UX Designer   
19                PT Asuransi Jiwa Astra            UI/UX Designer   

                                          
                                 company  
0   PT HZN Teknologi Indonesia (Horizon)  
1   PT HZN Teknologi Indonesia (Horizon)  
2                       Whello Indonesia  
3          DOKU, PT NUSA SATU INTI ARTHA  
4               PT Mobile Sarana Sentosa  
5                            Dipstrategy  
6                             Tinkerlust  
7                PT Nusa Satu Inti Artha  
8                               Dekoruma  
9    Nomura Research Institute Indonesia  
10                              Tokenomy  
11              NTT INDONESIA TECHNOLOGY  
12                                 Aleph  
13           Tera Logistik Internasional  
14                                  NOBI  
15                              Startups  
16          PT Majoo Teknologi Indonesia  
17     Mekari (PT. Mid Solusi Nusantara)  
18                  Bali Staff Solutions  
19                         1001malam.com

In [18]:
job_compare = pd.merge(df_compare['Minkowski'], df_compare['Cosine Similarity'], how = 'inner')
job_compare

,title,company
0,UX Designer,PT HZN Teknologi Indonesia (Horizon)
1,UI Designer - Internship,PT HZN Teknologi Indonesia (Horizon)
2,Senior UI/UX Designer,Tokenomy
3,UI/ UX Designer,Tinkerlust
4,UX / UI Designer,Startups
5,UI / UX Designer,PT Mobile Sarana Sentosa
6,UI/UX Designer,Mekari (PT. Mid Solusi Nusantara)
7,Senior UI/UX Designer,Whello Indonesia
8,Senior UI/UX Designer,PT Nusa Satu Inti Artha
9,Senior UI/UX Designer,"DOKU, PT NUSA SATU INTI ARTHA"


In [19]:
similarityPercentage = (job_compare.shape[0]/df_compare.shape[0])*100
print("Similarity between Minkowski Model and Cosine Similarity Model is {} %".format(similarityPercentage))

Similarity between Minkowski Model and Cosine Similarity Model is 65.0 %
